In [1]:
import mwu_measures
import pandas as pd
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher
mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
# this_corpus = mwu_measures.processing_corpus.get_processed_corpus('bnc', 'small_corpus.txt', chunk_size=10000000, verbose=False)
this_corpus = mwu_measures.processing_corpus.get_processed_corpus('coca', 'mwu_measures/corpora/coca_sample/', chunk_size=3, verbose=True)

# this_corpus = mwu_measures.processing_corpus.process_corpus(test_corpus=True)

# ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)

# TODO WARNING unaggregated table uses A LOT of ram. Aggregate after each chunk.
# TODO: regex cleaning is slowing everything down. Problem seems to be on excessive wildcard use. Benchmark against simplified patterns?
# bigrams = [['b', 'd'], ['c', 'b'], ['a', 'c']]

helper = Fetcher(this_corpus)

['text_acad_1990.txt', 'text_acad_1991.txt', 'text_blog_18.txt']
4732
Timer unit: 1e-09 s

Total time: 7.38525 s
File: /home/pablo/Documents/GitHub/MWU_measures/mwu_measures/preprocessing_corpus.py
Function: clean_coca_lines at line 35

Line #      Hits         Time  Per Hit   % Time  Line Contents
    35                                           def clean_coca_lines(raw_lines, corpus_ids):
    36         1     909467.0 909467.0      0.0      raw_lines = pd.DataFrame({'lines': raw_lines, 'ids': corpus_ids})
    37         1 2055757278.0    2e+09     27.8      raw_lines['lines'] = raw_lines['lines'].str.replace(r'(?:@ )+| ?</?[ph]> ?| ?<br> ? | [\.\?\!] ', '#----SPLIT----#', regex=True)
    38         1   29185994.0    3e+07      0.4      raw_lines['lines'] = raw_lines['lines'].str.lower()
    39         1  423573363.0    4e+08      5.7      raw_lines['lines'] = raw_lines['lines'].str.replace(r' \n| &gt;|@@\d+', '', regex=True)
    40         1  241413427.0    2e+08      3.3      raw_li

AttributeError: 'float' object has no attribute 'split'

In [ ]:
ngrams = mwu_examples['Expression'].tolist()
helper.create_scores(ngrams)

In [ ]:
test_weights = {'token_freq': 1/8, 'dispersion': 1/4, 'type_1': 1/16, 'type_2': 1/16, 'entropy_1': 1/16, 'entropy_2': 1/16, 'fw_assoc': 1/8, 'bw_assoc': 1/4}

In [ ]:
helper.get_score_batch(mwu_examples['Expression']).sort_values(by='mwu_score')

In [ ]:
ngrams = ['come in handy', 'family friend', 'database management system', 'one hundred percent', 'line of control', 'like a', 'boiled potatoes']

In [ ]:
self = helper
normalized=True
self.get_score_batch(ngrams)

In [ ]:

sentence = 'Hello! Come in and eat boiled potatoes they are one hundred percent good.\nI remember when we met in the year 2000'
sentence = mwu_measures.process_text(sentence)

In [ ]:
self.create_scores(sentence)

In [ ]:
self.get_score_batch(sentence)

In [ ]:
# TODO: Rewrite documentation and merge onto main. This is basically ready.
# TODO: CoCA corpus support. GOOD..
# TODO: Brown as default corpus.
# TODO: 4-grams. Should need minimal modification. Challenge might be RAM. Consider implementing an option to work from disk with duckdb?
# TODO: batches to not crash memory when using complete corpus. Worth it? faster because of the simpler joins? worth it to hash? Maybe a way to add rows to a table? Test whether it crashes with a long list of ngrams or with CoCA in the future.

In [ ]:
import os
coca_dir = 'mwu_measures/corpora/coca_texts/'
import pandas as pd
import re
import itertools
#from mwu_measures.preprocessing_corpus import preprocess_corpus
from mwu_measures.preprocessing_corpus import clean_coca_lines

coca_texts = sorted(os.listdir(coca_dir))
corpus_ids = dict(zip(sorted(coca_texts), range(len(coca_texts))))

In [ ]:
coca_chunks = [coca_texts[i:i+20] for i in range(0, len(coca_texts), 20)]

In [ ]:
for coca_chunk in coca_chunks:
    lines_chunk = []
    ids_chunk = []
    for coca_text in coca_chunk:
        with open(os.path.join(coca_dir, coca_text)) as corpus_file:
            raw_lines = corpus_file.readlines()
            line_chunks.extend(raw_lines)
            id_chunks.extend([corpus_ids[coca_text]] * len(raw_lines))


In [ ]:
x = pd.DataFrame({'lines': line_chunks, 'ids': id_chunks})

In [ ]:
x['split_lines'] = x['lines'].str.split(r'(?:@ )+| ?</?[ph]> ?| ?<br> ?', regex=True)
x = x.explode('split_lines')

In [ ]:
x = x.explode('lines')

In [ ]:
x

In [ ]:
from mwu_measures.preprocessing_corpus import preprocess_corpus
preprocess_corpus(raw_lines, 'coca', 1)

In [ ]:
raw_lines = pd.Series(raw_lines).str.split(r'(?:@ )+| ?</?[ph]> ?| ?<br> ? | [\.\?\!] ', regex=True).explode(ignore_index=True) # Splits paragraphs, lines, and sentences
raw_lines = raw_lines.str.strip()
raw_lines = raw_lines.str.lower()
raw_lines = raw_lines.str.replace(r' \n| &gt;|@@\d+', '', regex=True)
raw_lines = raw_lines.str.replace(r" (n't|'s|'ll|'d|'re|'ve|'m)", r"\1", regex=True)
raw_lines = raw_lines.str.replace('wan na', 'wanna', regex = False)
raw_lines = raw_lines.str.replace('-', '', regex=False)
raw_lines = raw_lines.str.replace(r'\s\d+\s|^\d+\s|\s\d+$', ' NUMBER ', regex=True)
raw_lines = raw_lines.str.strip()
raw_lines = raw_lines.str.replace(r'\s+\W\s+|\s+\W$|\W\s+$', ' ', regex=True)
raw_lines = raw_lines.str.strip()
raw_lines = raw_lines.str.replace(r'\s+', ' ', regex=True)
raw_lines = raw_lines[raw_lines.str.len() > 0]
raw_lines = 'START ' + raw_lines + ' END'
lines = list(zip(itertools.cycle([corpus_ids[this_corpus]]), raw_lines.to_list(), ))

In [ ]:
    y = x.str.replace(r'^.+\t', '', regex=True)
    y = y.str.lower()
    y = y.str.replace(r" (n't|'s|'ll|'d|'re|'ve|'m)", r"\1", regex=True)
    y = y.str.replace('wan na', 'wanna', regex = False)
    y = y.str.replace('\n', '')
    y = y.str.replace('-', '')
    y = y.str.replace(r'\s\d+\s|^\d+\s|\s\d+$', ' NUMBER ', regex=True)
    y = y.str.strip()
    y = y.str.replace(r'\s*\W+\s*', ' ', regex=True)
    y = y.str.strip()
    y = y.str.replace(r'\s+', ' ', regex=True)
    y = 'START ' + y + ' END'

delete:  \n| &gt;|@@\d+ 

In [ ]:
re.split(, raw_lines[7])

In [ ]:
raw_lines

In [ ]:
with open('mwu_measures/corpora/coca_texts/', 'r', encoding="utf-8") as corpus_file:
    i = 0
    while True:
        raw_lines = corpus_file.readlines(chunk_size)
        if not raw_lines:
            break
        ngram_dicts = preprocessing_corpus.preprocess_bnc(raw_lines)
        this_corpus.add_chunk(ngram_dicts)
        if verbose:
            i += len(raw_lines)
            print(f'{i} lines processed')